In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from ipywidgets import interact, widgets, Layout

stoppested=pd.read_csv("stops.txt", header=0, sep=",")
def finnNavn(stop_id):
    return stoppested[stoppested["stop_id"]==stop_id]["stop_name"].values[0]

##Leser inn datasett
df4=pd.read_csv("4eren_uke5.csv", header=0)

##konverterer tider til datetime
df4["departureTime"] = pd.to_datetime(df4["departureTime"])
df4["arrivalTime"] = pd.to_datetime(df4["arrivalTime"])
df4["aimedArrivalTime"] = pd.to_datetime(df4["aimedArrivalTime"])
df4["aimedDepartureTime"] = pd.to_datetime(df4["aimedDepartureTime"])
df4["operatingDate"] = pd.to_datetime(df4["operatingDate"])


## legger til avvik
df4["delayArrival"]=df4["arrivalTime"]-df4["aimedArrivalTime"] # for første stopp i ruten vil denne være NaT
df4["delayDeparture"]=df4["departureTime"]-df4["aimedDepartureTime"] # for siste stopp i ruten vil denne være NaT

##legger til rutetidUtenDato: (altså forventet avgangsttid, hvis den er null så forventet ankomststid (siste stopp i ruten))
df4["rutetidUtenDato"] = np.where(
    pd.isnull(df4["aimedDepartureTime"]),
    df4["aimedArrivalTime"].dt.time,
    df4["aimedDepartureTime"].dt.time
)





In [ ]:
df6 = df4.copy()
df6.dropna(subset=["delayDeparture", "delayArrival"], how='all', inplace=True) 
df6.sort_values(by=["aimedArrivalTime"], inplace=True)  
alle_avganger = df6["serviceJourneyId"].unique()
test_avganger = alle_avganger[:10]
maxindex = len(alle_avganger)-1
##print(maxindex)

def plottAvgang(i):
    serviceJourneyId=alle_avganger[i]
    avgang=df4[df4["serviceJourneyId"]==serviceJourneyId] #henter ut en enkeltavgang
    avgang=avgang[["stopPointRef","delayDeparture","sequenceNr","rutetidUtenDato"]].groupby("sequenceNr").agg({"delayDeparture":['max','mean','min'],"stopPointRef":"first", "rutetidUtenDato":"first"}).reset_index()#grupperer avgangen etter stopp langs ruten, der sequencenr gir rekkefølgen på stoppene
    avgang.columns=["sequenceNr","maxDelayDeparture","meanDelayDeparture","minDelayDeparture","stopPointRef","rutetidUtenDato"]
    avgang["maxDelayDeparture"]=avgang["maxDelayDeparture"].dt.total_seconds()/60#konverterer til minutter
    avgang["meanDelayDeparture"]=avgang["meanDelayDeparture"].dt.total_seconds()/60
    avgang["minDelayDeparture"]=avgang["minDelayDeparture"].dt.total_seconds()/60

    ##legger til navn på stoppested
    avgang["stopName"]=avgang["stopPointRef"].apply(finnNavn)
    

    xverdier=avgang["sequenceNr"]

    avgang["label"]=avgang.apply(lambda row: f"{row['rutetidUtenDato']} - {row['stopName']}", axis=1)
    print("Avgang: ",avgang["label"].values[0]," til ",avgang["label"].values[-1])
    
    plt.plot(xverdier,avgang["meanDelayDeparture"], label="meanDelayDeparture")
    plt.plot(xverdier,avgang["maxDelayDeparture"], label="maxDelayDeparture")
    plt.plot(xverdier,avgang["minDelayDeparture"], label="minDelayDeparture")
    plt.legend()
    plt.xlabel("Stopp")
    plt.xticks(labels=avgang["label"], ticks=xverdier, rotation=85)
    plt.ylabel("Forsinkelse")
    plt.title("Forsinkelse per stopp på avgang "+serviceJourneyId)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.show
    

interact(plottAvgang,i= widgets.IntSlider(value=0, min=0, max=maxindex,step=1,layout=Layout(width='500px')), continuous_update=False)



interactive(children=(IntSlider(value=0, description='i', layout=Layout(width='500px'), max=476), Output()), _…

<function __main__.plottAvgang(i)>